In [1]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import sys
sys.path.append( '../' )
import importlib

import measures_common
importlib.reload(measures_common)

<module 'measures_common' from 'd:\\Projetos\\TCC\\Part_2.1-Complex_Networks\\adjacency\\..\\measures_common.py'>

In [2]:
file_path_with_stopwords = os.path.join('..', 'csvs', 'pre_normalized_with_stopwords_measures.csv')
file_path_without_stopwords = os.path.join('..', 'csvs', 'pre_normalized_measures.csv')

df_with_stopwords = pd.read_csv(file_path_with_stopwords)
df_without_stopwords = pd.read_csv(file_path_without_stopwords)

df_with_stopwords.head(3)

,text,label,betweenness,closeness,eigenvector,katz,pagerank,hubs,authorities,clustering,average_clustering,correlation,transitivity,density
0,autor bestseller cita 5 expressoes indicam int...,fake,0.053793,0.107789,0.054755,0.078508,0.006211,0.006211,0.006211,0.016841,0.016841,-0.041276,0.015000,0.008579
1,juiz df confirma indicios expresidente petista...,fake,0.194226,0.065108,0.076284,0.116096,0.013514,0.013514,0.013514,0.023574,0.023574,-0.082239,0.047619,0.015920
2,senhora 60 anos corre atras eduardo cunha aero...,fake,0.163834,0.137540,0.160614,0.182358,0.033333,0.033333,0.033333,0.013889,0.013889,0.010157,0.050000,0.041379


In [3]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

NB = GaussianNB()
DT = DecisionTreeClassifier()
RF = RandomForestClassifier()
KNN = KNeighborsClassifier()
SVM = SVC()
MLP = MLPClassifier(max_iter=1000)

algs = [NB, DT, RF, KNN, SVM, MLP]
algs_names = ['NB', 'DT', 'RF', 'KNN', 'SVM', 'MLP']

In [4]:
from collections import defaultdict
from sklearn.model_selection import cross_val_score

scores_single_with_stopwords = defaultdict(list)
scores_single_without_stopwords = defaultdict(list)
measures = df_with_stopwords.columns[2:]

for alg, alg_name in tqdm(zip(algs, algs_names), total=len(algs), position=0, leave=True, desc="Single measure"):
    for measure in tqdm(measures, total=len(measures), position=1, leave=True, desc=alg_name):
        scores_single_with_stopwords["measure"].append(measure)
        scores_single_with_stopwords["algorithm"].append(alg_name)
        scores_single_with_stopwords["accuracy"].append(cross_val_score(alg, df_with_stopwords[measure].values.reshape(-1, 1), df_with_stopwords['label'], cv=5, scoring='accuracy').mean())

        scores_single_without_stopwords["measure"].append(measure)
        scores_single_without_stopwords["algorithm"].append(alg_name)
        scores_single_without_stopwords["accuracy"].append(cross_val_score(alg, df_without_stopwords[measure].values.reshape(-1, 1), df_without_stopwords['label'], cv=5, scoring='accuracy').mean())

Single measure:   0%|          | 0/6 [00:00<?, ?it/s]

NB:   0%|          | 0/12 [00:00<?, ?it/s]

DT:   0%|          | 0/12 [00:00<?, ?it/s]

RF:   0%|          | 0/12 [00:00<?, ?it/s]

KNN:   0%|          | 0/12 [00:00<?, ?it/s]

SVM:   0%|          | 0/12 [00:00<?, ?it/s]

MLP:   0%|          | 0/12 [00:00<?, ?it/s]

In [5]:
df_single_with_stopwords = pd.DataFrame(scores_single_with_stopwords)
df_single_without_stopwords = pd.DataFrame(scores_single_without_stopwords)

df_single = pd.merge(df_single_with_stopwords, df_single_without_stopwords, on=['measure', 'algorithm'], suffixes=('_with_stopwords', '_without_stopwords')) 
df_single = df_single.set_index(['measure', 'algorithm'])

cols = pd.MultiIndex.from_product([["with stopwords","without stopwords"], ['accuracy']])
df_single = pd.DataFrame(df_single[["accuracy_with_stopwords", "accuracy_without_stopwords"]].values, columns=cols, index=df_single.index)
df_single = df_single.sort_values(by=('with stopwords', 'accuracy'), ascending=False)

In [7]:
save_path = os.path.join('..', 'results', 'compare_models_single_adjacency.csv')

df_single = df_single.reset_index(col_level=1)

df_single.to_csv(save_path, index=False)

In [5]:
from collections import defaultdict
from sklearn.model_selection import cross_val_score

scores_all_with_stopwords = defaultdict(list)
scores_all_without_stopwords = defaultdict(list)
measures = df_with_stopwords.columns[2:]

for alg, alg_name in tqdm(zip(algs, algs_names), total=len(algs), position=0, leave=True, desc="All measures"):
    scores_all_with_stopwords["algorithm"].append(alg_name)
    scores_all_with_stopwords["accuracy"].append(cross_val_score(alg, df_with_stopwords[measures], df_with_stopwords['label'], cv=5, scoring='accuracy').mean())

    scores_all_without_stopwords["algorithm"].append(alg_name)
    scores_all_without_stopwords["accuracy"].append(cross_val_score(alg, df_without_stopwords[measures], df_without_stopwords['label'], cv=5, scoring='accuracy').mean())

All measures:   0%|          | 0/6 [00:00<?, ?it/s]

In [34]:
df_all_with_stopwords = pd.DataFrame(scores_all_with_stopwords)
df_all_without_stopwords = pd.DataFrame(scores_all_without_stopwords)

df_all = pd.merge(df_all_with_stopwords, df_all_without_stopwords, on='algorithm', suffixes=('_with_stopwords', '_without_stopwords'))
df_all = df_all.set_index('algorithm')

cols = pd.MultiIndex.from_product([["with stopwords","without stopwords"], ['accuracy']])
df_all = pd.DataFrame(df_all[["accuracy_with_stopwords", "accuracy_without_stopwords"]].values, columns=cols, index=df_all.index)
df_all = df_all.sort_values(by=('with stopwords', 'accuracy'), ascending=False)

,with stopwords,without stopwords
,accuracy,accuracy
algorithm,,
MLP,58.62%,58.69%
SVM,57.31%,57.35%
RF,56.69%,57.07%
NB,55.11%,55.12%
KNN,54.40%,54.37%
DT,53.00%,53.07%


In [75]:
read_path = os.path.join('..', 'results', 'compare_models_single_adjacency.csv')
df_single = pd.read_csv(read_path, header=[0,1], index_col=[0,1])
df_single.index.names = ['measure', 'algorithm']
df_single = pd.DataFrame(df_single.values, columns=pd.MultiIndex.from_product([["with stopwords","without stopwords"], ['accuracy']]), index=df_single.index)

In [78]:
df_single.loc[df_single.groupby(level=0).idxmax().values[:,1]].style.highlight_max(props='color:lightgreen;', axis=0)

,,with stopwords,without stopwords
,,accuracy,accuracy
measure,algorithm,,
authorities,SVM,0.508889,0.508889
average_clustering,KNN,0.504306,0.504306
betweenness,NB,0.509722,0.509722
closeness,SVM,0.542083,0.542083
clustering,KNN,0.504306,0.504306
correlation,MLP,0.532778,0.533889
density,RF,0.522639,0.524306
eigenvector,SVM,0.505833,0.507639


In [79]:
read_path = os.path.join('..', 'results', 'compare_models_all_adjacency.csv')

df_all = pd.read_csv(read_path, header=[0,1], index_col=0)

df_all.style.format("{:.2%}").highlight_max(props='color:lightgreen;')

,with stopwords,without stopwords
algorithm,accuracy,accuracy
MLP,58.62%,58.69%
SVM,57.31%,57.35%
RF,56.69%,57.07%
NB,55.11%,55.12%
KNN,54.40%,54.37%
DT,53.00%,53.07%
